In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

In [2]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.util import ngrams
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bwils\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bwils\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bwils\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
injury = pd.read_csv('severeinjury.csv', encoding='latin-1')

C:\Users\bwils\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


We are going to look at OSHA recorded severe injuries between January 2015 and Spetmeber 2020. Interested to see if there is a pattern in what injuries lead to hospitalization and or amputation.

In [4]:
injury.head()

,ID,UPA,EventDate,Employer,Address1,Address2,City,State,Zip,Latitude,...,Nature,NatureTitle,Part of Body,Part of Body Title,Event,EventTitle,Source,SourceTitle,Secondary Source,Secondary Source Title
0,2015010015,931176,1/1/2015,FCI Otisville Federal Correctional Institution,Two Mile Drive,NaN,OTISVILLE,NEW YORK,10963.0,41.46,...,111,Fractures,513,Lower leg(s),1214,Injured by physical contact with person while ...,5721,Co-worker,5772.0,Inmate or detainee in custody
1,2015010016,930267,1/1/2015,Kalahari Manufacturing LLC,171 Progress Drive,NaN,LAKE DELTON,WISCONSIN,53940.0,43.59,...,1522,Second degree heat (thermal) burns,519,"Leg(s), n.e.c.",317,"Ignition of vapors, gases, or liquids",7261,"Welding, cutting, and blow torches",NaN,NaN
2,2015010018,929823,1/1/2015,Schneider National Bulk Carrier,420 CORAOPOLIS ROAD,NaN,CORAOPOLIS,PENNSYLVANIA,15108.0,40.49,...,10,"Traumatic injuries and disorders, unspecified",9999,Nonclassifiable,4331,Other fall to lower level less than 6 feet,8421,"Semi, tractor-trailer, tanker truck",741.0,Ladders-fixed
3,2015010019,929711,1/1/2015,PEPSI BOTTLING GROUP INC.,4541 HOUSTON AVE.,NaN,MACON,GEORGIA,31206.0,32.77,...,1972,"Soreness, pain, hurt-nonspecified injury",510,"Leg(s), unspecified",640,Caught in or compressed by equipment or object...,8623,Pallet jack-powered,8420.0,"Truck-motorized freight hauling and utility, u..."
4,2015010020,929642,1/1/2015,North American Pipe Corporation,210 South Arch Street,NaN,JANESVILLE,WISCONSIN,53545.0,42.67,...,111,Fractures,4429,"Finger(s), fingernail(s), n.e.c.",6411,Caught in running equipment or machinery durin...,350,"Metal, woodworking, and special material machi...",NaN,NaN


There are a number of columns that we can elminate to make the data more manageable. Since most labor laws are made on the state and federal level we will drop location data except for state. Further inspection could be done on the dropped columns, but we are going to focus our search for now. The source data noted the the Lat and Long columns may be unreliable so we will be dropping them.

We can also drop the UPA ID as it is a duplicate ID, and the Final Narrative. We could run NLP on the FInal Narrative, but that is beyond the scope of this review. 

We will also drop titles columns after constructing dictionaries for labeling them later.

In [5]:
injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59469 entries, 0 to 59468
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      59469 non-null  int64  
 1   UPA                     59469 non-null  int64  
 2   EventDate               59469 non-null  object 
 3   Employer                59469 non-null  object 
 4   Address1                59457 non-null  object 
 5   Address2                5488 non-null   object 
 6   City                    59457 non-null  object 
 7   State                   59469 non-null  object 
 8   Zip                     59455 non-null  float64
 9   Latitude                59441 non-null  float64
 10  Longitude               59441 non-null  float64
 11  Primary NAICS           59467 non-null  object 
 12  Hospitalized            59469 non-null  float64
 13  Amputation              59467 non-null  float64
 14  Inspection              20129 non-null

Before we drop them we will first fill in our missing fields. 
- Because we don't intend to use them in our analysis we will ignore the following: Address1, Address2, City, Zip, Latitude, Longitude. 
- Primary NAICS will be filled with 0 as will hospitalization and amputation. 
- Inspections will be converted to a binary column as the report numbers don't provide us with further information.
- Secondary Souce will be filled with 0 and Secondary Source Title will be filled with None

In [6]:
fill = {'Primary NAICS':0, 'Hospitalized':0, 'Amputation':0, 'Inspection':0, 
        'Secondary Source':0, 'Secondary Source Title':'None'}
injury.fillna(value=fill, inplace=True)

In [7]:
injury.loc[injury['Inspection']!=0, 'Inspection'] = 1
injury.head()

,ID,UPA,EventDate,Employer,Address1,Address2,City,State,Zip,Latitude,...,Nature,NatureTitle,Part of Body,Part of Body Title,Event,EventTitle,Source,SourceTitle,Secondary Source,Secondary Source Title
0,2015010015,931176,1/1/2015,FCI Otisville Federal Correctional Institution,Two Mile Drive,NaN,OTISVILLE,NEW YORK,10963.0,41.46,...,111,Fractures,513,Lower leg(s),1214,Injured by physical contact with person while ...,5721,Co-worker,5772.0,Inmate or detainee in custody
1,2015010016,930267,1/1/2015,Kalahari Manufacturing LLC,171 Progress Drive,NaN,LAKE DELTON,WISCONSIN,53940.0,43.59,...,1522,Second degree heat (thermal) burns,519,"Leg(s), n.e.c.",317,"Ignition of vapors, gases, or liquids",7261,"Welding, cutting, and blow torches",0.0,None
2,2015010018,929823,1/1/2015,Schneider National Bulk Carrier,420 CORAOPOLIS ROAD,NaN,CORAOPOLIS,PENNSYLVANIA,15108.0,40.49,...,10,"Traumatic injuries and disorders, unspecified",9999,Nonclassifiable,4331,Other fall to lower level less than 6 feet,8421,"Semi, tractor-trailer, tanker truck",741.0,Ladders-fixed
3,2015010019,929711,1/1/2015,PEPSI BOTTLING GROUP INC.,4541 HOUSTON AVE.,NaN,MACON,GEORGIA,31206.0,32.77,...,1972,"Soreness, pain, hurt-nonspecified injury",510,"Leg(s), unspecified",640,Caught in or compressed by equipment or object...,8623,Pallet jack-powered,8420.0,"Truck-motorized freight hauling and utility, u..."
4,2015010020,929642,1/1/2015,North American Pipe Corporation,210 South Arch Street,NaN,JANESVILLE,WISCONSIN,53545.0,42.67,...,111,Fractures,4429,"Finger(s), fingernail(s), n.e.c.",6411,Caught in running equipment or machinery durin...,350,"Metal, woodworking, and special material machi...",0.0,None


In [8]:
injury.columns

Index(['ID', 'UPA', 'EventDate', 'Employer', 'Address1', 'Address2', 'City',
       'State', 'Zip', 'Latitude', 'Longitude', 'Primary NAICS',
       'Hospitalized', 'Amputation', 'Inspection', 'Final Narrative', 'Nature',
       'NatureTitle', 'Part of Body', 'Part of Body Title', 'Event',
       'EventTitle', 'Source', 'SourceTitle', 'Secondary Source',
       'Secondary Source Title'],
      dtype='object')

In [9]:
injury.drop(columns=['ID', 'UPA','Employer', 'Address1', 'Address2','Latitude', 'Longitude', 
                      'Nature', 'Part of Body', 'Event', 'Source', 'Secondary Source', 
                     'Primary NAICS',] ,inplace=True)
injury.columns

Index(['EventDate', 'City', 'State', 'Zip', 'Hospitalized', 'Amputation',
       'Inspection', 'Final Narrative', 'NatureTitle', 'Part of Body Title',
       'EventTitle', 'SourceTitle', 'Secondary Source Title'],
      dtype='object')

# Narrative Top Words

In [10]:
injury['Final Narrative']

0        Three correctional facility guards were escort...
1        Employee in the Machine Shop received second d...
2        A truck driver fell approximately 4 feet while...
3        An employee's leg was pinned between a truck a...
4        An employee working on the Line 6 Auto-Beller ...
                               ...                        
59464    A postal employee fell down a flight of 5 stai...
59465    A trainer was training a new employee on how t...
59466    The grinding blade on a deactivated meat grind...
59467    An employee fell while climbing a poplar tree,...
59468    An employee was climbing from one catwalk to a...
Name: Final Narrative, Length: 59469, dtype: object

In [11]:
nar1 = injury['Final Narrative'][0]
nar1

"Three correctional facility guards were escorting a restrained federal prison inmate when he became disruptive, requiring the use of force. \nTwo guards and the inmate fell onto the Lieutenant's right leg, fracturing his fibula. He was transported to the hospital and released the following day."

In [12]:
sents = nltk.sent_tokenize(nar1)
words = nltk.word_tokenize(nar1)
unique_tokens = set(words)
average_tokens = round(len(words)/len(sents))
print('Sentences: {}'.format(len(sents)))
print('Words: {}'.format(len(words)))
print('Unique Words: {}'.format(len(unique_tokens)))
print('Average Words per Sentence: {}'.format(average_tokens))

Sentences: 3
Words: 51
Unique Words: 41
Average Words per Sentence: 17


In [13]:
stop_words = set(stopwords.words('english'))
final_tokens=[]
for each in words:
    if each not in stop_words:
        final_tokens.append(each)
print('Non Stop Words: {}'.format(len(final_tokens)))

Non Stop Words: 36


In [14]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in final_tokens]
%pprint
lemmatized_words

Pretty printing has been turned OFF


['Three', 'correctional', 'facility', 'guard', 'escort', 'restrain', 'federal', 'prison', 'inmate', 'become', 'disruptive', ',', 'require', 'use', 'force', '.', 'Two', 'guard', 'inmate', 'fell', 'onto', 'Lieutenant', "'s", 'right', 'leg', ',', 'fracture', 'fibula', '.', 'He', 'transport', 'hospital', 'release', 'follow', 'day', '.']

In [15]:
%pprint

Pretty printing has been turned ON


In [16]:
def prep_narrative(narrative):
    stop_words = set(stopwords.words('english'))|set(string.punctuation)
    sents = nltk.sent_tokenize(narrative)
    prepped_narrative = []
    for sentence in sents:
        words = nltk.word_tokenize(narrative)
        final_tokens=[]
        for each in words:
            if each.lower() not in stop_words:
                lemma = lemmatizer.lemmatize(each.lower(), pos='v') 
                final_tokens.append(lemma)
        prepped_narrative.extend(final_tokens)
    return prepped_narrative

In [17]:
prepped = prep_narrative(injury['Final Narrative'][0])
prepped

['three',
 'correctional',
 'facility',
 'guard',
 'escort',
 'restrain',
 'federal',
 'prison',
 'inmate',
 'become',
 'disruptive',
 'require',
 'use',
 'force',
 'two',
 'guard',
 'inmate',
 'fell',
 'onto',
 'lieutenant',
 "'s",
 'right',
 'leg',
 'fracture',
 'fibula',
 'transport',
 'hospital',
 'release',
 'follow',
 'day',
 'three',
 'correctional',
 'facility',
 'guard',
 'escort',
 'restrain',
 'federal',
 'prison',
 'inmate',
 'become',
 'disruptive',
 'require',
 'use',
 'force',
 'two',
 'guard',
 'inmate',
 'fell',
 'onto',
 'lieutenant',
 "'s",
 'right',
 'leg',
 'fracture',
 'fibula',
 'transport',
 'hospital',
 'release',
 'follow',
 'day',
 'three',
 'correctional',
 'facility',
 'guard',
 'escort',
 'restrain',
 'federal',
 'prison',
 'inmate',
 'become',
 'disruptive',
 'require',
 'use',
 'force',
 'two',
 'guard',
 'inmate',
 'fell',
 'onto',
 'lieutenant',
 "'s",
 'right',
 'leg',
 'fracture',
 'fibula',
 'transport',
 'hospital',
 'release',
 'follow',
 'day']

In [18]:
injury['lemmatized'] = injury['Final Narrative'].apply(prep_narrative)
#injury['lemmatized'] = pd.read_pickle('Narrative_lemmatized.pkl')

In [19]:
injury['lemmatized']

0        [three, correctional, facility, guard, escort,...
1        [employee, machine, shop, receive, second, deg...
2        [truck, driver, fell, approximately, 4, feet, ...
3        [employee, 's, leg, pin, truck, power, pallet,...
4        [employee, work, line, 6, auto-beller, reach, ...
                               ...                        
59464    [postal, employee, fell, flight, 5, stairs, su...
59465    [trainer, train, new, employee, use, crimp, ma...
59466    [grind, blade, deactivate, meat, grind, machin...
59467    [employee, fell, climb, poplar, tree, land, gr...
59468    [employee, climb, one, catwalk, another, emplo...
Name: lemmatized, Length: 59469, dtype: object

In [20]:
injury['lemmatized'].to_pickle('Narrative_lemmatized.pkl')

In [21]:
def ranked_words(row):
    frequent = nltk.FreqDist(row)
    return frequent.most_common(5)

def place_words(row, rank):
    frequent = nltk.FreqDist(row)
    if len(frequent) > rank:
        return frequent.most_common(5)[rank-1][0]
    else:
        return None

In [22]:
injury['top_words'] = injury['lemmatized'].apply(ranked_words)

In [23]:
injury['top_words']

0        [(guard, 6), (inmate, 6), (three, 3), (correct...
1        [(employee, 1), (machine, 1), (shop, 1), (rece...
2        [(truck, 2), (driver, 2), (fell, 2), (approxim...
3        [(employee, 4), ('s, 2), (leg, 2), (pin, 2), (...
4        [(employee, 1), (work, 1), (line, 1), (6, 1), ...
                               ...                        
59464    [(postal, 1), (employee, 1), (fell, 1), (fligh...
59465    [(trainer, 4), (employee, 4), (machine, 4), (t...
59466    [(grind, 2), (blade, 1), (deactivate, 1), (mea...
59467    [(femur, 4), (employee, 2), (fell, 2), (climb,...
59468    [(employee, 4), (climb, 2), (one, 2), (catwalk...
Name: top_words, Length: 59469, dtype: object

In [24]:
for i in range(1,6):
    injury['top_word_{}'.format(i)] = injury['lemmatized'].apply(lambda x: place_words(x, i))

In [25]:
injury

,EventDate,City,State,Zip,Hospitalized,Amputation,Inspection,Final Narrative,NatureTitle,Part of Body Title,EventTitle,SourceTitle,Secondary Source Title,lemmatized,top_words,top_word_1,top_word_2,top_word_3,top_word_4,top_word_5
0,1/1/2015,OTISVILLE,NEW YORK,10963.0,1.0,0.0,1.0,Three correctional facility guards were escort...,Fractures,Lower leg(s),Injured by physical contact with person while ...,Co-worker,Inmate or detainee in custody,"[three, correctional, facility, guard, escort,...","[(guard, 6), (inmate, 6), (three, 3), (correct...",guard,inmate,three,correctional,facility
1,1/1/2015,LAKE DELTON,WISCONSIN,53940.0,1.0,0.0,1.0,Employee in the Machine Shop received second d...,Second degree heat (thermal) burns,"Leg(s), n.e.c.","Ignition of vapors, gases, or liquids","Welding, cutting, and blow torches",None,"[employee, machine, shop, receive, second, deg...","[(employee, 1), (machine, 1), (shop, 1), (rece...",employee,machine,shop,receive,second
2,1/1/2015,CORAOPOLIS,PENNSYLVANIA,15108.0,1.0,0.0,0.0,A truck driver fell approximately 4 feet while...,"Traumatic injuries and disorders, unspecified",Nonclassifiable,Other fall to lower level less than 6 feet,"Semi, tractor-trailer, tanker truck",Ladders-fixed,"[truck, driver, fell, approximately, 4, feet, ...","[(truck, 2), (driver, 2), (fell, 2), (approxim...",truck,driver,fell,approximately,4
3,1/1/2015,MACON,GEORGIA,31206.0,1.0,0.0,1.0,An employee's leg was pinned between a truck a...,"Soreness, pain, hurt-nonspecified injury","Leg(s), unspecified",Caught in or compressed by equipment or object...,Pallet jack-powered,"Truck-motorized freight hauling and utility, u...","[employee, 's, leg, pin, truck, power, pallet,...","[(employee, 4), ('s, 2), (leg, 2), (pin, 2), (...",employee,'s,leg,pin,truck
4,1/1/2015,JANESVILLE,WISCONSIN,53545.0,1.0,0.0,1.0,An employee working on the Line 6 Auto-Beller ...,Fractures,"Finger(s), fingernail(s), n.e.c.",Caught in running equipment or machinery durin...,"Metal, woodworking, and special material machi...",None,"[employee, work, line, 6, auto-beller, reach, ...","[(employee, 1), (work, 1), (line, 1), (6, 1), ...",employee,work,line,6,auto-beller
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59464,9/30/2020,PIQUA,OH,45356.0,1.0,0.0,0.0,A postal employee fell down a flight of 5 stai...,Internal injuries to organs and blood vessels ...,"Abdomen, except internal location of diseases ...","Other fall to lower level, unspecified","Stairs, steps, unspecified","Floors, walkways, ground surfaces, unspecified","[postal, employee, fell, flight, 5, stairs, su...","[(postal, 1), (employee, 1), (fell, 1), (fligh...",postal,employee,fell,flight,5
59465,9/30/2020,CLEARWATER,FL,33760.0,0.0,1.0,0.0,A trainer was training a new employee on how t...,Amputations,Fingertip(s),"Caught in running equipment or machinery, n.e.c.","Bending, crimping machines",None,"[trainer, train, new, employee, use, crimp, ma...","[(trainer, 4), (employee, 4), (machine, 4), (t...",trainer,employee,machine,train,new
59466,9/30/2020,MILWAUKEE,WI,53203.0,0.0,1.0,0.0,The grinding blade on a deactivated meat grind...,Amputations,Fingertip(s),Struck against moving part of machinery or equ...,Meat grinders,None,"[grind, blade, deactivate, meat, grind, machin...","[(grind, 2), (blade, 1), (deactivate, 1), (mea...",grind,blade,deactivate,meat,machine
59467,9/30/2020,LEBANON,PA,17046.0,1.0,0.0,1.0,"An employee fell while climbing a poplar tree,...",Fractures,"Multiple body parts, n.e.c.",Other fall to lower level 26 to 30 feet,Trees,"Floors, walkways, ground surfaces, unspecified","[employee, fell, climb, poplar, tree, land, gr...","[(femur, 4), (employee, 2), (fell, 2), (climb,...",femur,employee,fell,climb,poplar


In [26]:
injury.isnull().sum()

EventDate                   0
City                       12
State                       0
Zip                        14
Hospitalized                0
Amputation                  0
Inspection                  0
Final Narrative             0
NatureTitle                 0
Part of Body Title          0
EventTitle                  0
SourceTitle                 0
Secondary Source Title      0
lemmatized                  0
top_words                   0
top_word_1                  1
top_word_2                  3
top_word_3                 47
top_word_4                225
top_word_5                586
dtype: int64

In [27]:
injury.fillna({'City':'unknown', 'Zip':0}, inplace=True)

In [28]:
injury['EventDate'] = pd.to_datetime(injury['EventDate'])
injury['Zip'] = injury['Zip'].astype('int')
injury.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59469 entries, 0 to 59468
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   EventDate               59469 non-null  datetime64[ns]
 1   City                    59469 non-null  object        
 2   State                   59469 non-null  object        
 3   Zip                     59469 non-null  int32         
 4   Hospitalized            59469 non-null  float64       
 5   Amputation              59469 non-null  float64       
 6   Inspection              59469 non-null  float64       
 7   Final Narrative         59469 non-null  object        
 8   NatureTitle             59469 non-null  object        
 9   Part of Body Title      59469 non-null  object        
 10  EventTitle              59469 non-null  object        
 11  SourceTitle             59469 non-null  object        
 12  Secondary Source Title  59469 non-null  object

# State Names

In [29]:
state_mapper = {'NY':'NEW YORK', 'WI':'WISCONSIN', 'PA':'PENNSYLVANIA', 'GA':'GEORGIA', 
                'FL':'FLORIDA', 'CO':'COLORADO', 'OK':'OKLAHOMA', 'TX':'TEXAS', 'LA':'LOUISIANA',
                'MI':'MISSISSIPPI','NJ':'NEW JERSEY', 'OH':'OHIO', 'IL':'ILLINOIS', 'NE':'NEBRASKA',
                'NH':'NEW HAMPSHIRE', 'KS':'KANSAS', 'MA':'MASSACHUSETTS', 'AR':'ARKANSAS', 
                'MI':'MICHIGAN', 'ID':'IDAHO', 'MI':'MISSOURI', 'ME':'MAINE', 'CT':'CONNECTICUT',
                'WV':'WEST VIRGINIA', 'ND':'NORTH DAKOTA', 'MT':'MONTANA', 'NC':'NORTH CAROLINA',
                'DE':'DELAWARE', 'CA':'CALIFORNIA', 'DC':'DISTRICT OF COLUMBIA', 'AL':'ALABAMA',
                'TN':'TENNESSEE', 'OR':'OREGON', 'SD':'SOUTH DAKOTA', 'RI':'RHODE ISLAND', 
                'IN':'INDIANA', 'VA':'VIRGINIA', 'NM':'NEW MEXICO', 'MD':'MARYLAND', 'UT':'UTAH', 
                'VT':'VERMONT', 'AZ':'ARIZONA', 'IA':'IOWA', 'KY':'KENTUCKY', 'MN':'MINNESOTA',
                'WA':'WASHINGTON', 'SC':'SOUTH CAROLINA', 'HI':'HAWAII', 'PR':'PUERTO RICO', 
                'VI':'VIRGIN ISLANDS','GU':'GUAM', 'NV':'NEVADA', 'WY':'WYOMING', 
                'AK':'ALASKA', 'NMI':'NORTHERN MARIANA ISLANDS', 'AS':'AMERICAN SAMOA', 'MO':'MISSOURI',
               'MS':'MISSISSIPPI'} 

injury['State'].replace(state_mapper, inplace=True)
injury['State'].unique()

array(['NEW YORK', 'WISCONSIN', 'PENNSYLVANIA', 'GEORGIA', 'FLORIDA',
       'COLORADO', 'OKLAHOMA', 'TEXAS', 'LOUISIANA', 'MISSISSIPPI',
       'NEW JERSEY', 'OHIO', 'ILLINOIS', 'NEBRASKA', 'NEW HAMPSHIRE',
       'KANSAS', 'MASSACHUSETTS', 'ARKANSAS', 'MICHIGAN', 'IDAHO',
       'MISSOURI', 'MAINE', 'CONNECTICUT', 'WEST VIRGINIA',
       'NORTH DAKOTA', 'MONTANA', 'NORTH CAROLINA', 'DELAWARE',
       'CALIFORNIA', 'AMERICAN SAMOA', 'DISTRICT OF COLUMBIA', 'ALABAMA',
       'TENNESSEE', 'OREGON', 'SOUTH DAKOTA', 'RHODE ISLAND', 'INDIANA',
       'VIRGINIA', 'NEW MEXICO', 'MARYLAND', 'UTAH', 'VERMONT', 'ARIZONA',
       'GUAM', 'IOWA', 'KENTUCKY', 'MINNESOTA', 'WASHINGTON',
       'SOUTH CAROLINA', 'HAWAII', 'PUERTO RICO', 'VIRGIN ISLANDS',
       'NEVADA', 'WYOMING', 'NORTHERN MARIANA ISLANDS', 'ALASKA'],
      dtype=object)

# Part of Body

In [30]:
injury['Part of Body Title'].unique()

array(['Lower leg(s)', 'Leg(s), n.e.c.', 'Nonclassifiable',
       'Leg(s), unspecified', 'Finger(s), fingernail(s), n.e.c.',
       'Elbow(s)', 'Fingertip(s)', 'Upper and lower limb(s)',
       'BODY SYSTEMS', 'Hip(s)', 'Multiple body parts, n.e.c.',
       'Foot (feet), unspecified', 'Arm(s), unspecified', 'Wrist(s)',
       'Back, including spine, spinal cord, unspecified', 'Forearm(s)',
       'Ankle(s) and leg(s), unspecified',
       'Neck, except internal location of diseases or disorders',
       'Toes(s), toenail(s)', 'Thigh(s)', 'Brain',
       'Ankle(s) and leg(s), n.e.c.',
       'Finger(s), fingernail(s), unspecified', 'Ankle(s)', 'Spleen',
       'Face, unspecified', 'Arm(s), n.e.c.', 'Hand(s), unspecified',
       'Chest, except internal location of diseases or disorders',
       'Knee(s)',
       'Abdomen, except internal location of diseases or disorders',
       'Hand(s) and finger(s)', 'Hand(s), n.e.c.',
       'Foot(feet) and knee(s)', 'Lumbar region',
       'Hand(

In [102]:
injury['Part of Body Title Short'] = injury['Part of Body Title'].copy()
injury.loc[injury['Part of Body Title'].str.contains('foot|feet|toe|Foot|Feet|toe|heel|Heel|sole|Sole|Arch|instep|Ankle'), 'Part of Body Title Short'] = 'Foot'
injury.loc[injury['Part of Body Title'].str.contains('finger|hand|Finger|Hand|Wrist'), 'Part of Body Title Short'] = 'Hand'
injury.loc[injury['Part of Body Title'].str.contains('knee|leg|Leg|Knee|Butt|butt|Lower extremities|Thigh'), 'Part of Body Title Short'] = 'Leg'
injury.loc[injury['Part of Body Title'].str.contains('Arm|arm|Elbow|elbow|shoulder|Shoulder|Upper extremities'), 'Part of Body Title Short'] = 'Arm'
injury.loc[injury['Part of Body Title'].str.contains('Head|head|Face|face|Mouth|mouth|Nose|nose|Eye|Ear|Brain|Lip|Skull|Scalp|Tooth|Cranial|Cheek|Jaw'), 'Part of Body Title Short'] = 'Head'
injury.loc[injury['Part of Body Title'].str.contains('Back|back|Lumbar'), 'Part of Body Title Short'] = 'Back'
injury.loc[injury['Part of Body Title'].str.contains('Trunk|trunk|hip|Hip|Chest|chest'), 'Part of Body Title Short'] = 'Core'
injury.loc[injury['Part of Body Title'].str.contains('Multiple|multiple|Whole Body|Upper and Lower'), 'Part of Body Title Short'] = 'Multiple'
injury.loc[injury['Part of Body Title'].str.contains('organ|Internal|Organ|internal|Lung|Liver|Spleen|Heart|Thoracic|Cocc|Sacral'), 'Part of Body Title Short'] = 'Organ'
injury.loc[injury['Part of Body Title'].str.contains('Pelv|Groin|Testis|Scrotum'), 'Part of Body Title Short'] = 'Groin'
injury['Part of Body Title Short'].unique().sort()
injury['Part of Body Title Short'].unique()

array(['Leg', 'Nonclassifiable', 'Hand', 'Arm', 'Upper and lower limb(s)',
       'BODY SYSTEMS', 'Core', 'Multiple', 'Foot', 'Back', 'Organ',
       'Head', 'Groin', 'Whole body'], dtype=object)

# Right to Work

In [32]:
rtw = ['Alabama', 'Arizona', 'Arkansas', 'Florida', 'Georgia', 'Idaho', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 
       'Louisiana', 'Michigan','Mississippi', 'Nebraska', 'Nevada', 'North Carolina', 'North Dakota', 'Oklahoma', 
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia' 'West Virginia', 'Wisconsin', 'Wyoming']
for index in range(len(rtw)):
    rtw[index] = rtw[index].upper()
injury['RTW'] = False
injury.loc[injury['State'].isin(rtw), 'RTW'] = True

In [33]:
injury['RTW']

0        False
1         True
2        False
3         True
4         True
         ...  
59464    False
59465     True
59466     True
59467    False
59468     True
Name: RTW, Length: 59469, dtype: bool

# Presidential Voting

In [34]:
prez_red = ['Alaska', 'Alabama', 'Arkansas', 'Florida', 'Idaho', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 
       'Louisiana', 'Missouri','Mississippi', 'Montana', 'Nebraska', 'North Carolina', 'North Dakota', 'Ohio','Oklahoma', 
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'West Virginia', 'Wyoming']
for index in range(len(prez_red)):
    prez_red[index] = prez_red[index].upper()
injury['prez_red'] = 'Democrat'
injury.loc[injury['State'].isin(prez_red), 'prez_red'] = 'Republican'

In [35]:
injury

,EventDate,City,State,Zip,Hospitalized,Amputation,Inspection,Final Narrative,NatureTitle,Part of Body Title,...,lemmatized,top_words,top_word_1,top_word_2,top_word_3,top_word_4,top_word_5,Part of Body Title Short,RTW,prez_red
0,2015-01-01,OTISVILLE,NEW YORK,10963,1.0,0.0,1.0,Three correctional facility guards were escort...,Fractures,Lower leg(s),...,"[three, correctional, facility, guard, escort,...","[(guard, 6), (inmate, 6), (three, 3), (correct...",guard,inmate,three,correctional,facility,Leg,False,Democrat
1,2015-01-01,LAKE DELTON,WISCONSIN,53940,1.0,0.0,1.0,Employee in the Machine Shop received second d...,Second degree heat (thermal) burns,"Leg(s), n.e.c.",...,"[employee, machine, shop, receive, second, deg...","[(employee, 1), (machine, 1), (shop, 1), (rece...",employee,machine,shop,receive,second,Leg,True,Democrat
2,2015-01-01,CORAOPOLIS,PENNSYLVANIA,15108,1.0,0.0,0.0,A truck driver fell approximately 4 feet while...,"Traumatic injuries and disorders, unspecified",Nonclassifiable,...,"[truck, driver, fell, approximately, 4, feet, ...","[(truck, 2), (driver, 2), (fell, 2), (approxim...",truck,driver,fell,approximately,4,Nonclassifiable,False,Democrat
3,2015-01-01,MACON,GEORGIA,31206,1.0,0.0,1.0,An employee's leg was pinned between a truck a...,"Soreness, pain, hurt-nonspecified injury","Leg(s), unspecified",...,"[employee, 's, leg, pin, truck, power, pallet,...","[(employee, 4), ('s, 2), (leg, 2), (pin, 2), (...",employee,'s,leg,pin,truck,Leg,True,Democrat
4,2015-01-01,JANESVILLE,WISCONSIN,53545,1.0,0.0,1.0,An employee working on the Line 6 Auto-Beller ...,Fractures,"Finger(s), fingernail(s), n.e.c.",...,"[employee, work, line, 6, auto-beller, reach, ...","[(employee, 1), (work, 1), (line, 1), (6, 1), ...",employee,work,line,6,auto-beller,Hand,True,Democrat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59464,2020-09-30,PIQUA,OHIO,45356,1.0,0.0,0.0,A postal employee fell down a flight of 5 stai...,Internal injuries to organs and blood vessels ...,"Abdomen, except internal location of diseases ...",...,"[postal, employee, fell, flight, 5, stairs, su...","[(postal, 1), (employee, 1), (fell, 1), (fligh...",postal,employee,fell,flight,5,Organ,False,Republican
59465,2020-09-30,CLEARWATER,FLORIDA,33760,0.0,1.0,0.0,A trainer was training a new employee on how t...,Amputations,Fingertip(s),...,"[trainer, train, new, employee, use, crimp, ma...","[(trainer, 4), (employee, 4), (machine, 4), (t...",trainer,employee,machine,train,new,Hand,True,Republican
59466,2020-09-30,MILWAUKEE,WISCONSIN,53203,0.0,1.0,0.0,The grinding blade on a deactivated meat grind...,Amputations,Fingertip(s),...,"[grind, blade, deactivate, meat, grind, machin...","[(grind, 2), (blade, 1), (deactivate, 1), (mea...",grind,blade,deactivate,meat,machine,Hand,True,Democrat
59467,2020-09-30,LEBANON,PENNSYLVANIA,17046,1.0,0.0,1.0,"An employee fell while climbing a poplar tree,...",Fractures,"Multiple body parts, n.e.c.",...,"[employee, fell, climb, poplar, tree, land, gr...","[(femur, 4), (employee, 2), (fell, 2), (climb,...",femur,employee,fell,climb,poplar,Multiple,False,Democrat


# Spending

### Public

In [36]:
public = pd.read_csv('slstate.csv')
public.head()

,Unnamed: 0,Unnamed: 1,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018r,2019,Unnamed: 16,Unnamed: 17
0,NaN,Connecticut,"2,204","2,583","2,403","2,836","2,530","2,572","2,689","2,828","2,877","3,304","3,242","2,398","2,695","2,822",NaN,NaN
1,NaN,Maine,706,511,669,887,"1,559","1,544",780,753,530,647,542,416,705,"1,401",NaN,NaN
2,NaN,Massachusetts,"4,125","3,643","4,660","5,020","4,825","5,518","6,354","6,339","6,437","7,346","6,260","5,891","5,985","6,685",NaN,NaN
3,NaN,New Hampshire,"1,160",770,789,764,668,655,569,495,492,751,871,885,887,609,NaN,NaN
4,NaN,Rhode Island,800,795,650,779,595,566,468,577,459,573,648,553,721,815,NaN,NaN


In [37]:
public.columns

Index(['Unnamed: 0', 'Unnamed: 1', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018r', '2019',
       'Unnamed: 16', 'Unnamed: 17'],
      dtype='object')

In [38]:
public.drop(columns=['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', 'Unnamed: 0', 'Unnamed: 16', 'Unnamed: 17'], inplace=True)
public.rename(columns={'Unnamed: 1':'state', '2018r':'2018'}, inplace=True)
public

,state,2015,2016,2017,2018,2019
0,Connecticut,"3,304","3,242","2,398","2,695","2,822"
1,Maine,647,542,416,705,"1,401"
2,Massachusetts,"7,346","6,260","5,891","5,985","6,685"
3,New Hampshire,751,871,885,887,609
4,Rhode Island,573,648,553,721,815
...,...,...,...,...,...,...
59,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
public.dropna(inplace=True)
public

,state,2015,2016,2017,2018,2019
0,Connecticut,"3,304","3,242","2,398","2,695","2,822"
1,Maine,647,542,416,705,"1,401"
2,Massachusetts,"7,346","6,260","5,891","5,985","6,685"
3,New Hampshire,751,871,885,887,609
4,Rhode Island,573,648,553,721,815
5,Vermont,279,253,519,615,214
6,New Jersey,"6,704","6,940","5,135","5,490","5,666"
7,New York,"21,448","20,434","20,392","21,217","24,048"
8,Pennsylvania,"9,280","8,406","7,470","8,141","9,596"
9,Illinois,"10,828","9,893","9,416","9,095","9,062"


In [40]:
public_state = public.set_index('state')
pub_stack = pd.DataFrame(public_state.stack(), columns=['public']).sort_index()
pub_stack

public
state              
Alabama 2015  3,569
        2016  3,392
        2017  2,874
        2018  3,360
        2019  3,723
...             ...
Wyoming 2015  1,590
        2016  1,472
        2017    971
        2018    876
        2019    771

[255 rows x 1 columns]

### Private

In [41]:
private = pd.read_csv('nrstate.csv')
private.head()

,Unnamed: 0,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2013,2014,2015,2016,2017,2018r,2019,RSE(%),Unnamed: 19,Unnamed: 20
0,Connecticut,"1,621","2,278","2,386","2,755","3,752","4,121","2,732","1,521","1,736",...,"2,225","1,868","1,920","1,933","2,011","2,325","2,274",7.4,NaN,NaN
1,Maine,565,604,738,777,"1,254","1,232",636,603,385,...,586,667,667,495,609,605,838,7.0,NaN,NaN
2,Massachusetts,"5,609","5,515","5,266","5,155","6,577","7,791","6,378","5,202","4,511",...,"6,252","6,331","8,032","8,868","11,080","11,720","11,111",3.0,NaN,NaN
3,New Hampshire,"1,146",882,838,"1,128","1,332","1,139",946,955,"1,049",...,679,616,564,627,893,713,684,11.0,NaN,NaN
4,Rhode Island,537,541,530,928,866,"1,003",817,512,460,...,268,325,500,530,874,"1,187","1,554",8.2,NaN,NaN


In [42]:
private.columns

Index(['Unnamed: 0', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018r',
       '2019', 'RSE(%)', 'Unnamed: 19', 'Unnamed: 20'],
      dtype='object')

In [43]:
private.drop(columns=['Unnamed: 19', 'Unnamed: 20', 'RSE(%)', '2003', '2004', '2005', '2006', '2007', 
                      '2008', '2009', '2010', '2011', '2012', '2013', '2014'], inplace=True)
private.rename(columns={'Unnamed: 0':'state', '2018r':'2018'}, inplace=True)
private

,state,2015,2016,2017,2018,2019
0,Connecticut,"1,920","1,933","2,011","2,325","2,274"
1,Maine,667,495,609,605,838
2,Massachusetts,"8,032","8,868","11,080","11,720","11,111"
3,New Hampshire,564,627,893,713,684
4,Rhode Island,500,530,874,"1,187","1,554"
...,...,...,...,...,...,...
59,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
private.dropna(inplace=True)
private

,state,2015,2016,2017,2018,2019
0,Connecticut,"1,920","1,933","2,011","2,325","2,274"
1,Maine,667,495,609,605,838
2,Massachusetts,"8,032","8,868","11,080","11,720","11,111"
3,New Hampshire,564,627,893,713,684
4,Rhode Island,500,530,874,"1,187","1,554"
5,Vermont,440,542,605,447,363
6,New Jersey,"4,340","4,833","6,475","6,982","6,725"
7,New York,"24,596","30,901","27,879","31,481","32,058"
8,Pennsylvania,"7,640","7,138","8,245","12,638","12,591"
9,Illinois,"9,976","9,748","8,799","8,989","7,567"


In [45]:
private_state = private.set_index('state')
priv_stack = pd.DataFrame(private_state.stack(), columns=['private']).sort_index()
priv_stack

private
state               
Alabama 2015   2,904
        2016   3,090
        2017   3,156
        2018   4,165
        2019   6,030
...              ...
Wyoming 2015     946
        2016     585
        2017     474
        2018     351
        2019     298

[255 rows x 1 columns]

In [46]:
pub_stack.index.rename(['state','year'], inplace=True)

In [49]:
pub_stack.insert(1, value=priv_stack['private'], column='private')
pub_stack

public private
state   year               
Alabama 2015  3,569   2,904
        2016  3,392   3,090
        2017  2,874   3,156
        2018  3,360   4,165
        2019  3,723   6,030
...             ...     ...
Wyoming 2015  1,590     946
        2016  1,472     585
        2017    971     474
        2018    876     351
        2019    771     298

[255 rows x 2 columns]

### Spending Ratio

In [50]:
spending = pub_stack.reset_index()
spending

,state,year,public,private
0,Alabama,2015,"3,569","2,904"
1,Alabama,2016,"3,392","3,090"
2,Alabama,2017,"2,874","3,156"
3,Alabama,2018,"3,360","4,165"
4,Alabama,2019,"3,723","6,030"
...,...,...,...,...
250,Wyoming,2015,"1,590",946
251,Wyoming,2016,"1,472",585
252,Wyoming,2017,971,474
253,Wyoming,2018,876,351


In [51]:
spending['public'] = spending['public'].str.replace(',', '')
spending['private'] = spending['private'].str.replace(',', '')
spending = spending.astype({'public':'int', 'private':'int'})
spending['year'] = pd.to_datetime(spending['year'])
spending['year'] = spending['year'].dt.year
spending

,state,year,public,private
0,Alabama,2015,3569,2904
1,Alabama,2016,3392,3090
2,Alabama,2017,2874,3156
3,Alabama,2018,3360,4165
4,Alabama,2019,3723,6030
...,...,...,...,...
250,Wyoming,2015,1590,946
251,Wyoming,2016,1472,585
252,Wyoming,2017,971,474
253,Wyoming,2018,876,351


In [52]:
spending.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   state    255 non-null    object
 1   year     255 non-null    int64 
 2   public   255 non-null    int32 
 3   private  255 non-null    int32 
dtypes: int32(2), int64(1), object(1)
memory usage: 6.1+ KB


In [53]:
spending['ratio'] = spending['private']/spending['public']
spending

,state,year,public,private,ratio
0,Alabama,2015,3569,2904,0.813673
1,Alabama,2016,3392,3090,0.910967
2,Alabama,2017,2874,3156,1.098121
3,Alabama,2018,3360,4165,1.239583
4,Alabama,2019,3723,6030,1.619662
...,...,...,...,...,...
250,Wyoming,2015,1590,946,0.594969
251,Wyoming,2016,1472,585,0.397418
252,Wyoming,2017,971,474,0.488157
253,Wyoming,2018,876,351,0.400685


In [55]:
spending['total'] = spending['private']+spending['public']
spending

,state,year,public,private,ratio,total
0,Alabama,2015,3569,2904,0.813673,6473
1,Alabama,2016,3392,3090,0.910967,6482
2,Alabama,2017,2874,3156,1.098121,6030
3,Alabama,2018,3360,4165,1.239583,7525
4,Alabama,2019,3723,6030,1.619662,9753
...,...,...,...,...,...,...
250,Wyoming,2015,1590,946,0.594969,2536
251,Wyoming,2016,1472,585,0.397418,2057
252,Wyoming,2017,971,474,0.488157,1445
253,Wyoming,2018,876,351,0.400685,1227


In [57]:
spending['state'] = spending['state'].str.upper()
spending

,state,year,public,private,ratio,total
0,ALABAMA,2015,3569,2904,0.813673,6473
1,ALABAMA,2016,3392,3090,0.910967,6482
2,ALABAMA,2017,2874,3156,1.098121,6030
3,ALABAMA,2018,3360,4165,1.239583,7525
4,ALABAMA,2019,3723,6030,1.619662,9753
...,...,...,...,...,...,...
250,WYOMING,2015,1590,946,0.594969,2536
251,WYOMING,2016,1472,585,0.397418,2057
252,WYOMING,2017,971,474,0.488157,1445
253,WYOMING,2018,876,351,0.400685,1227


In [60]:
injury['year']=injury['EventDate'].dt.year

In [62]:
injury['year']

0        2015
1        2015
2        2015
3        2015
4        2015
         ... 
59464    2020
59465    2020
59466    2020
59467    2020
59468    2020
Name: year, Length: 59469, dtype: int64

In [78]:
spending.rename(columns={'state':'State'}, inplace=True)
spending

,State,year,public,private,ratio,total
0,ALABAMA,2015,3569,2904,0.813673,6473
1,ALABAMA,2016,3392,3090,0.910967,6482
2,ALABAMA,2017,2874,3156,1.098121,6030
3,ALABAMA,2018,3360,4165,1.239583,7525
4,ALABAMA,2019,3723,6030,1.619662,9753
...,...,...,...,...,...,...
250,WYOMING,2015,1590,946,0.594969,2536
251,WYOMING,2016,1472,585,0.397418,2057
252,WYOMING,2017,971,474,0.488157,1445
253,WYOMING,2018,876,351,0.400685,1227


In [80]:
aggs = injury.groupby(['State','year'])[['EventDate', 'Hospitalized', 'Amputation']].agg({'EventDate':'count', 'Hospitalized':'sum', 
                                                                                    'Amputation':'sum'})
aggs = aggs.reset_index()
aggs.rename(columns={'EventDate':'injuries'}, inplace=True)
aggs

,State,year,injuries,Hospitalized,Amputation
0,ALABAMA,2015,305,235.0,108.0
1,ALABAMA,2016,324,256.0,100.0
2,ALABAMA,2017,302,243.0,88.0
3,ALABAMA,2018,393,333.0,112.0
4,ALABAMA,2019,362,296.0,112.0
...,...,...,...,...,...
321,WYOMING,2016,5,5.0,0.0
322,WYOMING,2017,3,3.0,0.0
323,WYOMING,2018,2,2.0,0.0
324,WYOMING,2019,2,1.0,1.0


In [83]:
combined = aggs.merge(spending, on=['State', 'year'], how='outer')
combined

,State,year,injuries,Hospitalized,Amputation,public,private,ratio,total
0,ALABAMA,2015,305.0,235.0,108.0,3569.0,2904.0,0.813673,6473.0
1,ALABAMA,2016,324.0,256.0,100.0,3392.0,3090.0,0.910967,6482.0
2,ALABAMA,2017,302.0,243.0,88.0,2874.0,3156.0,1.098121,6030.0
3,ALABAMA,2018,393.0,333.0,112.0,3360.0,4165.0,1.239583,7525.0
4,ALABAMA,2019,362.0,296.0,112.0,3723.0,6030.0,1.619662,9753.0
...,...,...,...,...,...,...,...,...,...
324,WYOMING,2019,2.0,1.0,1.0,771.0,298.0,0.386511,1069.0
325,WYOMING,2020,3.0,3.0,0.0,NaN,NaN,NaN,NaN
326,UTAH,2018,NaN,NaN,NaN,3789.0,2081.0,0.549221,5870.0
327,VERMONT,2018,NaN,NaN,NaN,615.0,447.0,0.726829,1062.0


In [84]:
# Right to Work

rtw = ['Alabama', 'Arizona', 'Arkansas', 'Florida', 'Georgia', 'Idaho', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 
       'Louisiana', 'Michigan','Mississippi', 'Nebraska', 'Nevada', 'North Carolina', 'North Dakota', 'Oklahoma', 
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia' 'West Virginia', 'Wisconsin', 'Wyoming']
for index in range(len(rtw)):
    rtw[index] = rtw[index].upper()
combined['RTW'] = False
combined.loc[combined['State'].isin(rtw), 'RTW'] = True

combined['RTW']

# Presidential Voting

prez_red = ['Alaska', 'Alabama', 'Arkansas', 'Florida', 'Idaho', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 
       'Louisiana', 'Missouri','Mississippi', 'Montana', 'Nebraska', 'North Carolina', 'North Dakota', 'Ohio','Oklahoma', 
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'West Virginia', 'Wyoming']
for index in range(len(prez_red)):
    prez_red[index] = prez_red[index].upper()
combined['prez_red'] = 'Democrat'
combined.loc[combined['State'].isin(prez_red), 'prez_red'] = 'Republican'

combined

,State,year,injuries,Hospitalized,Amputation,public,private,ratio,total,RTW,prez_red
0,ALABAMA,2015,305.0,235.0,108.0,3569.0,2904.0,0.813673,6473.0,True,Republican
1,ALABAMA,2016,324.0,256.0,100.0,3392.0,3090.0,0.910967,6482.0,True,Republican
2,ALABAMA,2017,302.0,243.0,88.0,2874.0,3156.0,1.098121,6030.0,True,Republican
3,ALABAMA,2018,393.0,333.0,112.0,3360.0,4165.0,1.239583,7525.0,True,Republican
4,ALABAMA,2019,362.0,296.0,112.0,3723.0,6030.0,1.619662,9753.0,True,Republican
...,...,...,...,...,...,...,...,...,...,...,...
324,WYOMING,2019,2.0,1.0,1.0,771.0,298.0,0.386511,1069.0,True,Republican
325,WYOMING,2020,3.0,3.0,0.0,NaN,NaN,NaN,NaN,True,Republican
326,UTAH,2018,NaN,NaN,NaN,3789.0,2081.0,0.549221,5870.0,True,Republican
327,VERMONT,2018,NaN,NaN,NaN,615.0,447.0,0.726829,1062.0,False,Democrat


In [85]:
remove = ['PUERTO RICO', 'VIRGIN ISLANDS','GUAM','NORTHERN MARIANA ISLANDS', 'AMERICAN SAMOA']
combined_clean = combined.loc[~combined['State'].isin(remove)]
combined_clean['State'].unique()

array(['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA',
       'COLORADO', 'CONNECTICUT', 'DELAWARE', 'DISTRICT OF COLUMBIA',
       'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA',
       'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND',
       'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI',
       'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE',
       'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA',
       'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA',
       'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE',
       'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON',
       'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'], dtype=object)

In [87]:
combined_clean['State'].value_counts()

HAWAII                  6
OKLAHOMA                6
MASSACHUSETTS           6
ARKANSAS                6
NORTH CAROLINA          6
KANSAS                  6
NEBRASKA                6
RHODE ISLAND            6
OHIO                    6
NEW JERSEY              6
UTAH                    6
WASHINGTON              6
NEVADA                  6
IDAHO                   6
NORTH DAKOTA            6
ARIZONA                 6
SOUTH CAROLINA          6
WYOMING                 6
MISSOURI                6
NEW HAMPSHIRE           6
COLORADO                6
ILLINOIS                6
CONNECTICUT             6
MICHIGAN                6
TENNESSEE               6
WEST VIRGINIA           6
DISTRICT OF COLUMBIA    6
SOUTH DAKOTA            6
INDIANA                 6
PENNSYLVANIA            6
NEW YORK                6
MONTANA                 6
IOWA                    6
FLORIDA                 6
MISSISSIPPI             6
GEORGIA                 6
VIRGINIA                6
MINNESOTA               6
ALASKA      

In [89]:
OSHA = combined_clean.fillna(0).copy()
OSHA

,State,year,injuries,Hospitalized,Amputation,public,private,ratio,total,RTW,prez_red
0,ALABAMA,2015,305.0,235.0,108.0,3569.0,2904.0,0.813673,6473.0,True,Republican
1,ALABAMA,2016,324.0,256.0,100.0,3392.0,3090.0,0.910967,6482.0,True,Republican
2,ALABAMA,2017,302.0,243.0,88.0,2874.0,3156.0,1.098121,6030.0,True,Republican
3,ALABAMA,2018,393.0,333.0,112.0,3360.0,4165.0,1.239583,7525.0,True,Republican
4,ALABAMA,2019,362.0,296.0,112.0,3723.0,6030.0,1.619662,9753.0,True,Republican
...,...,...,...,...,...,...,...,...,...,...,...
324,WYOMING,2019,2.0,1.0,1.0,771.0,298.0,0.386511,1069.0,True,Republican
325,WYOMING,2020,3.0,3.0,0.0,0.0,0.0,0.000000,0.0,True,Republican
326,UTAH,2018,0.0,0.0,0.0,3789.0,2081.0,0.549221,5870.0,True,Republican
327,VERMONT,2018,0.0,0.0,0.0,615.0,447.0,0.726829,1062.0,False,Democrat


In [105]:
pob = injury.groupby(['State', 'year'])['Part of Body Title Short'].value_counts().unstack(level=-1)
pob = pob.fillna(0).copy()
pob

Part of Body Title Short   Arm  BODY SYSTEMS  Back  Core  Foot  Groin   Hand  \
State   year                                                                   
ALABAMA 2015              18.0          16.0   6.0  15.0  24.0    1.0  129.0   
        2016              27.0          25.0   3.0  13.0  22.0    0.0  131.0   
        2017              18.0          13.0   4.0  15.0  19.0    1.0  124.0   
        2018              25.0          20.0   4.0  17.0  29.0    2.0  150.0   
        2019              24.0          23.0   8.0  16.0  28.0    5.0  137.0   
...                        ...           ...   ...   ...   ...    ...    ...   
WYOMING 2016               1.0           1.0   0.0   0.0   0.0    0.0    0.0   
        2017               1.0           0.0   0.0   0.0   0.0    0.0    0.0   
        2018               0.0           0.0   0.0   0.0   0.0    0.0    0.0   
        2019               0.0           0.0   0.0   1.0   0.0    0.0    1.0   
        2020               1.0           1.0   0.0   0.0   1.0    0.0    0.0   

Part of Body Title Short  Head   Leg  Multiple  Nonclassifiable  Organ  \
State   year                                                             
ALABAMA 2015              14.0  29.0      16.0             20.0   13.0   
        2016              16.0  36.0      32.0              4.0   14.0   
        2017              12.0  48.0      27.0              4.0   12.0   
        2018              17.0  51.0      39.0             14.0   22.0   
        2019              20.0  39.0      35.0             11.0   13.0   
...                        ...   ...       ...              ...    ...   
WYOMING 2016               1.0   0.0       1.0              1.0    0.0   
        2017               0.0   1.0       0.0              0.0    1.0   
        2018               0.0   1.0       0.0              1.0    0.0   
        2019               0.0   0.0       0.0              0.0    0.0   
        2020               0.0   0.0       0.0              0.0    0.0   

Part of Body Title Short  Upper and lower limb(s)  Whole body  
State   year                                                   
ALABAMA 2015                                  4.0         0.0  
        2016                                  1.0         0.0  
        2017                                  5.0         0.0  
        2018                                  3.0         0.0  
        2019                                  3.0         0.0  
...                                           ...         ...  
WYOMING 2016                                  0.0         0.0  
        2017                                  0.0         0.0  
        2018                                  0.0         0.0  
        2019                                  0.0         0.0  
        2020                                  0.0         0.0  

[326 rows x 14 columns]

In [106]:
pob.reset_index()

Part of Body Title Short,State,year,Arm,BODY SYSTEMS,Back,Core,Foot,Groin,Hand,Head,Leg,Multiple,Nonclassifiable,Organ,Upper and lower limb(s),Whole body
0,ALABAMA,2015,18.0,16.0,6.0,15.0,24.0,1.0,129.0,14.0,29.0,16.0,20.0,13.0,4.0,0.0
1,ALABAMA,2016,27.0,25.0,3.0,13.0,22.0,0.0,131.0,16.0,36.0,32.0,4.0,14.0,1.0,0.0
2,ALABAMA,2017,18.0,13.0,4.0,15.0,19.0,1.0,124.0,12.0,48.0,27.0,4.0,12.0,5.0,0.0
3,ALABAMA,2018,25.0,20.0,4.0,17.0,29.0,2.0,150.0,17.0,51.0,39.0,14.0,22.0,3.0,0.0
4,ALABAMA,2019,24.0,23.0,8.0,16.0,28.0,5.0,137.0,20.0,39.0,35.0,11.0,13.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,WYOMING,2016,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
322,WYOMING,2017,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
323,WYOMING,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
324,WYOMING,2019,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
OSHA = OSHA.merge(pob, on=['State', 'year'], how='outer')
OSHA

,State,year,injuries,Hospitalized,Amputation,public,private,ratio,total,RTW,...,Foot,Groin,Hand,Head,Leg,Multiple,Nonclassifiable,Organ,Upper and lower limb(s),Whole body
0,ALABAMA,2015,305.0,235.0,108.0,3569.0,2904.0,0.813673,6473.0,True,...,24.0,1.0,129.0,14.0,29.0,16.0,20.0,13.0,4.0,0.0
1,ALABAMA,2016,324.0,256.0,100.0,3392.0,3090.0,0.910967,6482.0,True,...,22.0,0.0,131.0,16.0,36.0,32.0,4.0,14.0,1.0,0.0
2,ALABAMA,2017,302.0,243.0,88.0,2874.0,3156.0,1.098121,6030.0,True,...,19.0,1.0,124.0,12.0,48.0,27.0,4.0,12.0,5.0,0.0
3,ALABAMA,2018,393.0,333.0,112.0,3360.0,4165.0,1.239583,7525.0,True,...,29.0,2.0,150.0,17.0,51.0,39.0,14.0,22.0,3.0,0.0
4,ALABAMA,2019,362.0,296.0,112.0,3723.0,6030.0,1.619662,9753.0,True,...,28.0,5.0,137.0,20.0,39.0,35.0,11.0,13.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,VIRGIN ISLANDS,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
325,VIRGIN ISLANDS,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
326,VIRGIN ISLANDS,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
327,VIRGIN ISLANDS,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0


In [108]:
OSHA = OSHA.loc[~OSHA['State'].isin(remove)]
OSHA

,State,year,injuries,Hospitalized,Amputation,public,private,ratio,total,RTW,...,Foot,Groin,Hand,Head,Leg,Multiple,Nonclassifiable,Organ,Upper and lower limb(s),Whole body
0,ALABAMA,2015,305.0,235.0,108.0,3569.0,2904.0,0.813673,6473.0,True,...,24.0,1.0,129.0,14.0,29.0,16.0,20.0,13.0,4.0,0.0
1,ALABAMA,2016,324.0,256.0,100.0,3392.0,3090.0,0.910967,6482.0,True,...,22.0,0.0,131.0,16.0,36.0,32.0,4.0,14.0,1.0,0.0
2,ALABAMA,2017,302.0,243.0,88.0,2874.0,3156.0,1.098121,6030.0,True,...,19.0,1.0,124.0,12.0,48.0,27.0,4.0,12.0,5.0,0.0
3,ALABAMA,2018,393.0,333.0,112.0,3360.0,4165.0,1.239583,7525.0,True,...,29.0,2.0,150.0,17.0,51.0,39.0,14.0,22.0,3.0,0.0
4,ALABAMA,2019,362.0,296.0,112.0,3723.0,6030.0,1.619662,9753.0,True,...,28.0,5.0,137.0,20.0,39.0,35.0,11.0,13.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,WYOMING,2019,2.0,1.0,1.0,771.0,298.0,0.386511,1069.0,True,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
301,WYOMING,2020,3.0,3.0,0.0,0.0,0.0,0.000000,0.0,True,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
302,UTAH,2018,0.0,0.0,0.0,3789.0,2081.0,0.549221,5870.0,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,VERMONT,2018,0.0,0.0,0.0,615.0,447.0,0.726829,1062.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
OSHA = OSHA.fillna(0).copy()
OSHA

,State,year,injuries,Hospitalized,Amputation,public,private,ratio,total,RTW,...,Foot,Groin,Hand,Head,Leg,Multiple,Nonclassifiable,Organ,Upper and lower limb(s),Whole body
0,ALABAMA,2015,305.0,235.0,108.0,3569.0,2904.0,0.813673,6473.0,True,...,24.0,1.0,129.0,14.0,29.0,16.0,20.0,13.0,4.0,0.0
1,ALABAMA,2016,324.0,256.0,100.0,3392.0,3090.0,0.910967,6482.0,True,...,22.0,0.0,131.0,16.0,36.0,32.0,4.0,14.0,1.0,0.0
2,ALABAMA,2017,302.0,243.0,88.0,2874.0,3156.0,1.098121,6030.0,True,...,19.0,1.0,124.0,12.0,48.0,27.0,4.0,12.0,5.0,0.0
3,ALABAMA,2018,393.0,333.0,112.0,3360.0,4165.0,1.239583,7525.0,True,...,29.0,2.0,150.0,17.0,51.0,39.0,14.0,22.0,3.0,0.0
4,ALABAMA,2019,362.0,296.0,112.0,3723.0,6030.0,1.619662,9753.0,True,...,28.0,5.0,137.0,20.0,39.0,35.0,11.0,13.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,WYOMING,2019,2.0,1.0,1.0,771.0,298.0,0.386511,1069.0,True,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
301,WYOMING,2020,3.0,3.0,0.0,0.0,0.0,0.000000,0.0,True,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
302,UTAH,2018,0.0,0.0,0.0,3789.0,2081.0,0.549221,5870.0,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303,VERMONT,2018,0.0,0.0,0.0,615.0,447.0,0.726829,1062.0,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
OSHA['ratio'].describe()

count    305.000000
mean       0.929305
std        0.645134
min        0.000000
25%        0.581114
50%        0.916233
75%        1.309811
max        3.820185
Name: ratio, dtype: float64

In [110]:
injury.to_csv('Injuries.csv')
spending.to_csv('Spending.csv')
OSHA.to_csv('OSHA.csv')